Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [297]:
# pip install yellowbrick

In [298]:
import plotly # для построения интерактивных визуализаций
import imblearn # для борьбы с проблемами несбалансированных наборов данных
import yellowbrick # ещё одна библиотека для визуализации...

import warnings
warnings.simplefilter('ignore')

In [299]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)
# df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [300]:
#Ваш код здесь
mw = df['gender'].value_counts()['Male'] / df['gender'].value_counts()['Female']
mw

1.0192087155963303

##### 2. Какое количество уникальных значений у поля InternetService?

In [301]:
#Ваш код здесь
df['InternetService'].value_counts().count()

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [302]:
#Ваш код здесь

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors = 'coerce')

print('median = ', df['TotalCharges'].median())
print('mean = ', df['TotalCharges'].mean())
print('std = ', df['TotalCharges'].std())

median =  1397.475
mean =  2283.3004408418697
std =  2266.7713618831453


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [303]:
#Ваш код здесь

df['PhoneService'] = df['PhoneService'].map({'Yes':'1', 'No':'0'}).astype(float)
df['PhoneService'].value_counts()

1.0    6361
0.0     682
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [304]:
#Ваш код здесь

df.loc[df['TotalCharges'] == ' ', 'TotalCharges'] = np.nan
df['TotalCharges'].fillna(0, inplace=True)

print('median = ', df['TotalCharges'].median())
print('mean = ', df['TotalCharges'].mean())
print('std = ', df['TotalCharges'].std())

median =  1394.55
mean =  2279.734303563826
std =  2266.7944696890195


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [305]:
#Ваш код здесь

df['Churn'] = df['Churn'].map({'Yes':'1', 'No':'0'}).astype(float)
df['Churn'].value_counts()

0.0    5174
1.0    1869
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [306]:
#Ваш код здесь

df['StreamingMovies'] = df['StreamingMovies'].map({'Yes':'1', 'No':'0', 'No internet service':'0'}).astype(float)
df['StreamingMovies'].value_counts()

0.0    4311
1.0    2732
Name: StreamingMovies, dtype: int64

In [307]:
df['StreamingTV'] = df['StreamingTV'].map({'Yes':'1', 'No':'0', 'No internet service':'0'}).astype(float)
df['StreamingTV'].value_counts()

0.0    4336
1.0    2707
Name: StreamingTV, dtype: int64

In [308]:
df['TechSupport'] = df['TechSupport'].map({'Yes':'1', 'No':'0', 'No internet service':'0'}).astype(float)
df['TechSupport'].value_counts()

0.0    4999
1.0    2044
Name: TechSupport, dtype: int64

##### 8. Заполните пропуски в поле PhoneService значением 0

In [309]:
#Ваш код здесь

df.loc[(df['PhoneService'].isnull()), ['PhoneService']] = 0
df['PhoneService'].value_counts()

1.0    6361
0.0     682
Name: PhoneService, dtype: int64

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [310]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь

df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0.0,29.85,0.0,0.0,0.0,0.0
1,Male,34,1.0,1889.50,0.0,0.0,0.0,0.0
2,Male,2,1.0,108.15,0.0,0.0,0.0,1.0


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [311]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], random_state=21)


In [312]:
### Попробовал простой RandomForest для сравнения...

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

X_train['gender'] = X_train['gender'].map({'Male':'1', 'Female':'0'}).astype(float)
X_test['gender'] = X_test['gender'].map({'Male':'1', 'Female':'0'}).astype(float)

model.fit(X_train, y_train)
preds = model.predict(X_test)

print('roc_auc = ', roc_auc_score(y_test, preds))
print('logloss = ', log_loss(y_test, preds))

roc_auc =  0.6472401279268233
logloss =  8.708325182781248


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [280]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [281]:
#Ваш код здесь

gender.fit(X_train, y_train)

Pipeline(steps=[('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))])

##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [282]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [283]:
#Ваш код здесь

TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [284]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [285]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [286]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [287]:
#Ваш код здесь

preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.13, 1.  , 0.38, ..., 0.61, 1.  , 1.  ])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [288]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь

print('roc_auc = ', roc_auc_score(y_test, preds))
print('logloss = ', log_loss(y_test, preds))

roc_auc =  0.7682196298076128
logloss =  0.8171730270973536


### Сохраним наш пайплайн

In [294]:
# pip install dill

In [295]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)